In [1]:
from acevedo_clss_and_fcns import * 
model                                 = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
feature_data                          = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
feature_names                         = pd.read_csv("./results/data/metabolite_names.csv")
grafo_nx             = cobra_to_networkx(model)
grafo_nx_onlyConcen  = add_metabolite_concentration_features(grafo_nx, feature_data, feature_names)
pyg_graph_onlyConcen = make_PYG_graph_from_grafo_nx(grafo_nx_onlyConcen)
labels                 = torch.tensor(feature_data.label).reshape(len(feature_data.label),1)
pyg_graph_onlyConcen.y = labels
#torch.save(pyg_graph_onlyConcen, "./results/graphs_from_PYG_and_NX/PYG_graph_only_Concen.pt")

/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [34]:
def make_graphs_list(pyg_graph_in,mask_target:bool=False, target_node:int = 0, mask_number = 0):
    
    pyg_graph = copy.deepcopy(pyg_graph_in)
    graphs_list = []
    
    
    for i in range(pyg_graph.x.shape[1]):
    
        
        new_pyg_data  = Data(x =  pyg_graph.x[:,i].reshape(pyg_graph.num_nodes, 1),  y = pyg_graph.y[i], 
                            edge_index = pyg_graph.edge_index)
        new_pyg_data.num_classes = 2
        if mask_target:
            new_pyg_data.x[target_node,:] = mask_number        
        
        graphs_list.append(new_pyg_data) 
    return graphs_list

class batch_loader():
    
    def __init__(self, graphs: list, batch_size: int  =1*32, num_samples:int = None, validation_percent:float = .3):
        self.graphs = graphs
        self.batch_size = batch_size
        self.num_samples = num_samples
        self.validation_percent = validation_percent
        self.train_idxs, self.val_idxs = train_test_split(range(len(self.graphs)), test_size = self.validation_percent)     
        
    def get_train_loader(self):
        train_subset = [self.graphs[i] for i in self.train_idxs]
        sampler      = RandomSampler(
        train_subset,
        #num_samples= self.num_samples, 
        replacement=False)   
        
        return  DataLoader(train_subset, batch_size= self.batch_size, sampler = sampler,  drop_last=True)
    
    def get_validation_loader(self):
        validation_subset = [self.graphs[i] for i in self.val_idxs]
        sampler      = RandomSampler(
        validation_subset,
        #num_samples= self.num_samples, 
        replacement=False)   
        
        return  DataLoader(validation_subset, batch_size= self.batch_size, sampler = sampler,  drop_last=True)

In [42]:
graphs_list_PYG_graph_only_Concen = make_graphs_list(pyg_graph_onlyConcen)

loader = batch_loader(graphs_list_PYG_graph_only_Concen, batch_size= 64, validation_percent = .35)


In [41]:
def make_loader(graphs: list, batch_size: int  =1*32, num_samples:int = None):#, num_samples:int = 20*32):#batch_size: int  =8*32):

    sampler_train_set = RandomSampler(
        graphs,
        #num_samples= num_samples, #params["training"]["sampler_num_samples"],  # Genera un muestreo del grafo
        replacement=False,  # con repeticion de muestras
    )
    return DataLoader(graphs, batch_size=batch_size, sampler = sampler_train_set,  drop_last=True)


loader_only_Concen   = make_loader(graphs_list_PYG_graph_only_Concen)
len(list(loader_only_Concen.dataset))

7572

In [43]:
gc.collect()
torch.cuda.empty_cache() 
#loader = loader_only_Concen
model    = GCN(hidden_channels=8)
optimizer     = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()
def train():
    model.to('cuda')
    model.train()

    for data in loader.get_train_loader():  # Iterate in batches over the training dataset.
         data.to('cuda')
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()
     model.to('cuda')
     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         data.to('cuda')
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 4):
    train()
    train_acc = test(loader.get_train_loader())
    test_acc = test(loader.get_validation_loader())
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6769, Test Acc: 0.6620
Epoch: 002, Train Acc: 0.9484, Test Acc: 0.9506
Epoch: 003, Train Acc: 0.9305, Test Acc: 0.9283


In [39]:
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda:0'

print(f"{device = }")
a_batch         = next(iter(loader.get_train_loader()))
a_graph         = a_batch[0]
model          = GIN_classifier(0, a_graph.num_nodes, a_graph.num_node_features)
optimizer     = torch.optim.Adam(model.parameters(), lr=0.01)
loss_function = torch.nn.NLLLoss()
best_val_model, last_best_state_dict_path, last_best_model_path =  train_and_validate(
                                    model,loss_function,optimizer, EPOCHS =4 ,
                                    train_loader= loader.get_train_loader(),
                                    validation_loader= loader.get_validation_loader(),
                                    save_state_dict = False,save_entire_model=False,
                                    verbose=True,
                                    saving_path = 'results', device=device)

device = 'cuda:0'


 25%|██▌       | 1/4 [00:21<01:04, 21.48s/it]

new best_val_model best_vloss = 0.9354960392304791


 25%|██▌       | 1/4 [00:26<01:19, 26.44s/it]


KeyboardInterrupt: 